In [1]:
import numpy as np
from scipy import signal
from pylab import *
import matplotlib.pyplot as plt

In [25]:
def initGEQ(G_db):
    fs = 44.1e3
    #fc1 = 1000 * 2 ** ( 1/3 * arange(-17,14))
    fc1 = np.array([19.69,24.80,31.25,39.37,49.61,62.50,78.75,99.21,125.0,157.5,198.4,
    250.0,315.0,396.9,500.0,630.0,793.7,1000,1260,1587,2000,2520,3175,4000,
    5040,6350,8000,10080,12700,16000,20160])
    fc2_list = [None] * (len(fc1)+len(sqrt(fc1[0:len(fc1)-1:1] * fc1[1::1])))
    fc2_list[::2] = fc1
    fc2_list[1::2] = sqrt(fc1[0:len(fc1)-1:1] * fc1[1::1])
    fc2 = np.array(fc2_list)
    wg = 2*pi*fc1/fs
    wc = 2*pi*fc2/fs
    c= 0.4
    bw = np.array((2 ** (1/3) - 2 ** (-1/3)) * wg)
    bw_adjust = 2*pi/fs
    # Bänder aus acge3.m
    bw[16::] = [370*bw_adjust, 466.2*bw_adjust, 587.4*bw_adjust, 740.1*bw_adjust, 932.4*bw_adjust, 1175*bw_adjust, 1480*bw_adjust, 1865*bw_adjust, 2350*bw_adjust, 2846*bw_adjust, 3502*bw_adjust, 4253*bw_adjust, 5038*bw_adjust, 5689*bw_adjust, 5570*bw_adjust]
    #bw[::] =[9.178*bw_adjust, 11.56*bw_adjust, 14.57*bw_adjust, 18.36*bw_adjust, 23.13*bw_adjust, 29.14*bw_adjust, 36.71*bw_adjust, 46.25*bw_adjust, 58.28*bw_adjust, 73.43*bw_adjust, 
    #92.51*bw_adjust, 116.6*bw_adjust, 146.9*bw_adjust, 185.0*bw_adjust, 233.1*bw_adjust, 293.7*bw_adjust,370*bw_adjust, 466.2*bw_adjust, 587.4*bw_adjust, 740.1*bw_adjust, 932.4*bw_adjust, 1175*bw_adjust, 1480*bw_adjust, 1865*bw_adjust, 2350*bw_adjust, 2846*bw_adjust, 3502*bw_adjust, 4253*bw_adjust, 5038*bw_adjust, 5689*bw_adjust, 5570*bw_adjust]
    # Bänder aus SGE initGEQ.m 
    #bw[16::] = [369.7*bw_adjust, 465.8*bw_adjust, 586.8*bw_adjust, 739.3*bw_adjust, 930.6*bw_adjust, 1172*bw_adjust, 1476*bw_adjust, 1857*bw_adjust, 2338*bw_adjust, 2943*bw_adjust, 3704*bw_adjust, 4638*bw_adjust, 5684*bw_adjust, 6803*bw_adjust, 4117*bw_adjust]
    leak = interactionMatrix(10**(17/20)*np.ones(31),c,wg,wc,bw)

    G_db2 = np.zeros([61,1])
    G_db2[::2] = G_db
    G_db2[1::2] = (G_db[:len(G_db)-1:1]+G_db[1::1])/2
    
    Gopt_db = np.linalg.lstsq(leak.conj().T, G_db2)[0]
    Gopt = 10**(Gopt_db/20)
    
    leak2 = interactionMatrix(Gopt,c,wg,wc,bw)
    G2opt_db = np.linalg.lstsq(leak2.conj().T, G_db2)[0]
    G2opt = 10 **(G2opt_db/20)
    G2wopt_db = c * G2opt_db
    G2wopt = 10 **(G2wopt_db/20)
    
    numsopt = np.zeros((3,31))
    densopt = np.zeros((3,31))
    
    for k in range(31):
        [num,den] = pareq(G2opt[k],G2wopt[k],wg[k],bw[k])
        numsopt[:,k] = num
        densopt[:,k] = den
        
    
    plot(numsopt,densopt,fs,fc2,G_db2)
    
    return numsopt,densopt

In [26]:
def pareq(G, GB, w0, B):
    if G == 1:
        beta = tan(B/2.)
    else: 
        beta = sqrt(abs(GB**2-1)/abs(G**2-GB**2))*tan(B/2)

    num = [(1+G*beta), -2*cos(w0), (1-G*beta)]/(1+beta)
    den = [1, -2*cos(w0)/(1+beta), (1-beta)/(1+beta)]
    
    return num, den

In [27]:
def interactionMatrix(G,c,wg,wc,bw):

    M = len(wg)
    N = len(wc)
    leak = np.zeros([M,N]) 
    Gdb = 20 * np.log10(G)
    Gdbw = c * Gdb
    Gw = 10 ** (Gdbw/20)
    
    for m in range(M):
        [num, den] = pareq(G[m],Gw[m],wg[m],bw[m])
        w,h = signal.freqz(num, den, wc)
        Gain = 20*np.log10(np.abs(h))/Gdb[m]
        leak[m,:] = np.abs(Gain)
    return leak

In [28]:
def plot(numsopt,densopt,fs,fc2,G_db2):
    N_freq = 2 **12
    w = np.logspace(np.log10(9),np.log10(22050), N_freq)
    H_opt = np.ones((N_freq,31), dtype=complex)
    H_opt_tot = np.ones((N_freq,1), dtype=complex)
    
    for k in range(31):
        w, h = signal.freqz(numsopt[:,k], densopt[:,k],worN=w,fs=fs)
        H_opt[:,k]= h
        H_opt_tot = H_opt[:,[k]]  * H_opt_tot
    
    
    plt.semilogx(w,20*np.log10(H_opt_tot))
    plt.plot(fc2,G_db2, "ro", markersize=4, markerfacecolor="none")
    plt.ylabel("Magnitude [dB]")
    plt.xlabel("Frequency [Hz]")
    plt.title("Frequency response")
    plt.xticks([10, 30, 100, 1000, 3000, 10000])
    plt.yticks(np.arange(-20,20,5))
    plt.grid(which="both", linestyle="--", color="grey")
    plt.show()

In [29]:
input = np.array([12,5,6,9,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,4,6,2])
input_alternative = np.array([12,-12,-12,12,-12,-12,12,-12,-12,12,-12,-12,12,-12,-12,12,-12,-12,12,-12,-12,12,-12,-12,12,-12,-12,12,-12,-12,12])    

In [30]:
b = initGEQ(input_alternative.reshape((-1,1)))  #reshape to make it 31x1 instead of 1x31

C:/Users/Robert/AppData/Local/Temp/xpython_15496/1632009434.py:28: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  Gopt_db = np.linalg.lstsq(leak.conj().T, G_db2)[0]
C:/Users/Robert/AppData/Local/Temp/xpython_15496/2647486580.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  num = [(1+G*beta), -2*cos(w0), (1-G*beta)]/(1+beta)
C:\Users\Robert\anaconda3\envs\ml\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays 